In [4]:
pip install accelerate==0.28.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip uninstall accelerate -y


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: accelerate 1.8.1
Uninstalling accelerate-1.8.1:
  Successfully uninstalled accelerate-1.8.1
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install evaluate

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torch transformers datasets scikit-learn accelerate


Note: you may need to restart the kernel to use updated packages.


In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset
import numpy as np
import evaluate

# STEP 1: Load Dataset
dataset = load_dataset("emotion")

# STEP 2: Load Tokenizer & Model
model_ckpt = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=6)

# STEP 3: Tokenize
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])  # remove raw text
tokenized_dataset.set_format("torch")

# STEP 4: Load Metric
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    preds = np.argmax(logits, axis=1)
    return accuracy.compute(predictions=preds, references=labels)

# STEP 5: Define Training Arguments
args = TrainingArguments(
    output_dir="./bert-tiny-emotion",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    load_best_model_at_end=True,
    report_to="none"
)

# STEP 6: Create Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# STEP 7: Train
trainer.train()

# STEP 8: Evaluate on validation set
eval_results = trainer.evaluate()
print("\n✅ Evaluation Results:", eval_results)

# STEP 9: Save model
model.save_pretrained("./bert-tiny-emotion")
tokenizer.save_pretrained("./bert-tiny-emotion")


/opt/conda/envs/anaconda-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/anaconda-env/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 2000/2000 [00:00<00:00, 8461.97 examples/s]
/opt/conda/envs/anaconda-env/lib/python3.12/site-packages/accelerate/accele

Epoch,Training Loss,Validation Loss,Accuracy
1,0.906600,0.782066,0.777500
2,0.561900,0.465810,0.868500
3,0.315800,0.374328,0.887500
4,0.331900,0.344148,0.899500
5,0.328400,0.335193,0.899500


/opt/conda/envs/anaconda-env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/conda/envs/anaconda-env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/conda/envs/anaconda-env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/conda/envs/anaconda-env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/conda/envs/anaconda-env/lib/python3


✅ Evaluation Results: {'eval_loss': 0.33519336581230164, 'eval_accuracy': 0.8995, 'eval_runtime': 4.0931, 'eval_samples_per_second': 488.622, 'eval_steps_per_second': 7.818, 'epoch': 5.0}


('./bert-tiny-emotion/tokenizer_config.json',
 './bert-tiny-emotion/special_tokens_map.json',
 './bert-tiny-emotion/vocab.txt',
 './bert-tiny-emotion/added_tokens.json',
 './bert-tiny-emotion/tokenizer.json')